In [34]:
import numpy as np 
import sys
sys.path.append("../")
import pickle as pkl
import os
from pathlib import Path
import fnmatch

from glob import glob
import search_world
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
# reading in scalar and vector data filenames for a specific path
job_id = ''
data_dir = os.path.join('../logs/', job_id)
config_fns = [fname for root, _, _ in os.walk(data_dir) for fname in glob(os.path.join(root, 'config.log'))]
scalar_fns = [fname for root, _, _ in os.walk(data_dir) for fname in glob(os.path.join(root, 'scalar'))]
vector_fns = [fname for root, _, _ in os.walk(data_dir) for fname in glob(os.path.join(root, 'vector'))]

In [19]:
# all together now
scalar_arrs = []
vector_dfs = []
scalar_dicts = []
for idx, (vector_fn, scalar_fn) in enumerate(zip(vector_fns, scalar_fns)):
    with open(scalar_fn, 'rb') as f:
        a = pkl.load(f)       
    scalar_dict = a['env'].info()    
    scalar_dict.update({'dataset_index': idx})
#     scalar_dict.update(a['model'].info())
    scalar_dicts.append(scalar_dict)

    with open(vector_fn, 'rb') as f:
        vector_dicts = pkl.load(f)
        flattened_vector_dicts = []        
        for d in vector_dicts:
            t = {}
            for k, v in d.items():
                if isinstance(v, dict):
                    for nested_k, nested_v in v.items():        
                        t.update({nested_k: nested_v})
                else:
                    t.update({k: v})      
            flattened_vector_dicts.append(t)
        df = pd.DataFrame(flattened_vector_dicts) 
        df['dataset_index'] = idx
        df['trial_index'] = df.done.cumsum().shift(fill_value=0)
        changes = df.columns.difference(['trial_index', 'dataset_index', 'job_id']).tolist()        

        f = lambda x: x.tolist() if len(x) > 1 else x

        df = df.groupby(['job_id', 'dataset_index', 'trial_index'])[changes].agg(f).reset_index()
        df['done'] = df.done.apply(lambda x: [x] if isinstance(x, bool) else x)
        df = df[df.apply(lambda x: True in x.done, axis=1)].reset_index()
        df.trial_index += idx
        vector_dfs.append(df)
scalar_df = pd.DataFrame(scalar_dicts)
df = pd.concat(vector_dfs).reset_index()
df = df.merge(scalar_df, on='dataset_index')

In [20]:
# TODO: new column for initial condition

In [21]:
condition = ['length', 'n_corridors', 'agent_initial_position', 'target_position']
df = df.set_index(condition) # grouping by initial condition and maze configuratio

In [46]:
# TODO: Find number of trials per condition
# TODO: Find agent score on each trial 
df['agent_dist'] = df.apply(lambda x: len(x.action), axis=1)
df['agent_score'] = 1 - (df.agent_dist - df.min_dist)/(df.max_dist - df.min_dist)
# TODO: Splitting dataframe into two datasets per condition
grouped_df = df.groupby(condition)
def mask(n, p=0.5):
    a = np.zeros(n, dtype=int)
    a[:int(p * n)] = 1
    np.random.shuffle(a)
    a = a.astype(bool)
    return a
masks = grouped_df.index.agg('count').agg(lambda x: mask(x)).to_frame()
df1 = grouped_df.apply(lambda x: x[masks.loc[x.index[0]][0]])
df2 = grouped_df.apply(lambda x: x[~masks.loc[x.index[0]][0]])
# TODO: Average scores across trials with the same condition
condition_scores_df1 = df1.groupby(['length', 'n_corridors', 'agent_initial_position', 'target_position']).agent_score.mean().to_frame()
condition_scores_df2 = df2.groupby(['length', 'n_corridors', 'agent_initial_position', 'target_position']).agent_score.mean().to_frame()
# TODO: Repeat this to do multi-step cross validation
# TODO: Compare average scores across trials of different agents
merged = condition_scores_df1.merge(condition_scores_df2, on=['length', 'n_corridors', 'agent_initial_position', 'target_position'])
print('MLS correlation': merged.agent_score_x.corr(merged.agent_score_y))

0.9088121286254445

In [55]:
print('no. of unique conditions: %d' % len(df.index.unique()))
print('avg. no. of trials per condition: %.2f ± %.2f' % (df.groupby(condition).index.count().mean(), df.groupby(condition).index.count().std()))

no. of unique conditions: 261
avg. no. of trials per condition: 51.71 ± 40.88
